# RAG
## 3. 파이콘(이라는 서버)에 저장    

https://python.langchain.com/v0.2/docs/tutorials/rag/  
1. 문서 내용 읽기
    - https://python.langchain.com/v0.2/docs/integrations/document_loaders/
1. 문서 분할(쪼개기)
    - 문서 분할하지 않으면?
        - 토큰 수 초과로 답변을 생성하지 않을 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 - > 답변 늦어짐
1. 임베딩 - 벡터 데이터베이스에 저장
1. 질문이 있으면 벡터 데이터베이스에 유사도 검색(코사인 유사도)
1. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader
from dotenv import load_dotenv


## 1, 2. 문서 Microsoft Word 파일 읽고 분할 한번에 
loader = Docx2txtLoader("law_1.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

data_list = loader.load_and_split(text_splitter=text_splitter)


# 3. 임베딩 -> 벡터 데이터베이스에 저장 
## 환경변수 읽어오기
load_dotenv()

## 3-1. 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)



In [15]:
%pip install langchain-pinecone
%pip install pinecone

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [ ]:

## 3-2 벡터 데이터베이스에 저장 
# ## [방법1] inmemory
# database = Chroma.from_documents(
#     documents=data_list,
#     embedding=embedding,
# )

## [방법2] 로컬에 저장(chroma 파일을 만들고 거기에 저장) 
# 한번 실행하면 생기기 때문에 두번 세번 할 필요가 없음.
# database = Chroma.from_documents(
#     documents=data_list,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law',
# )

# 2.1 두번째부터는 로컬에 저장된 임베딩 데이터 가져오기
# database = Chroma(
#     embedding_function=embedding,        
#     persist_directory='./chroma',                  
#     collection_name='chroma-law',
# )

## [방법3] 파이콘 서버에 저장 (두번세번하면 두번세번 같은게 추가 저장됨.)
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)


# 3.1 파이콘 DB에 데이터리스트를 인덱스 자동으로 붙게 해서 올림. 
# database = PineconeVectorStore.from_documents(
#     documents=data_list,
#     embedding=embedding, 
#     index_name='law-index',     
# )

# 3.2 올리는 거 없이 읽기만.
database = PineconeVectorStore(
    index_name='law-2-index',
    embedding=embedding,
)


database

In [12]:

## 4. 사용자 질문 + 벡터 데이터베이스에서 유사도 검색
query = "전세사기피해자 금융지원에 대해 설명해주세요."

## 벡터 데이터베이스에서 유사도 검색(기본이 4개씩)
retrieved_docs = database.similarity_search(query=query)


## 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 
## 5-1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기피해 법률 전문가입니다
-[context]를 참고해 사용자의 질문에 답변해주세요. 

[context]
{retrieved_docs} 

Question : {query}

'''

formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query,
)

## 5-2 LLM 모델 생성, ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model_name='gpt-4.1-nano'
)

aimessage = llm.invoke(formatted_prompt)
aimessage

AIMessage(content='전세사기피해자를 위한 금융지원은 법률과 정책에 따라 다양한 방식으로 제공되고 있습니다. 주요 내용은 다음과 같습니다.\n\n1. 긴급융자 및 주거안정 지원:\n   - 국가 및 지방자치단체는 전세사기피해자와 긴급한 주거안정을 필요로 하는 임차인에게 필요한 자금을 융자하거나 기타 지원을 할 수 있습니다. (제27조 ①)\n   - 「주택임대차보호법」에 따라 우선변제를 받지 못하는 위기상황에 있는 피해자에게는 주택도시기금에서 임대차에 필요한 자금을 융자하는 지원도 가능합니다. (제27조 ②)\n2. 채무 등록 유예 및 신용정보 관리:\n   - 금융회사(한국자산관리공사 포함)는 전세사기피해자의 보증금이 미변제된 경우, 신용정보집중기관을 통해 채무의 등록을 유예하거나 삭제할 수 있으며, 이는 피해자의 신용회복과 금융 접근성을 돕기 위한 조치입니다. (제27조 ③)\n3. 보증금 회수 및 채권 정리 지원:\n   - 피해자가 임차보증금에서 대항력 또는 우선변제권을 행사하거나 임차권등기를 통해 보증금을 회수한 경우, 이를 토대로 채무 변제 과정에서 지원이 이루어질 수 있습니다. (제28조 관련)\n4. 공공임대주택 및 재정 지원:\n   - 피해자가 전세사기피해로 인해 임차 보증금을 회수하지 못한 경우, 공공주택사업자와 협력하여 공공임대주택으로의 이주를 지원하며, 임대료 지원 및 차액 보전도 가능합니다. (제28조, 신설 2024. 9. 10.)\n5. 기타 정책적 지원:\n   - 피해자 지원을 위해 저금리 융자, 재정 지원, 임대료 보조 등 다양한 금융 지원 정책이 시행될 예정을 포함하고 있으며, 이러한 조치는 피해자의 주거 안정을 위해 지속적으로 확대될 예정입니다.\n\n이와 같이 정부와 관련 기관은 전세사기 피해자의 금전적 손실 보전과 주거 안정화를 위해 다양한 금융지원 제도를 마련하고 시행 중입니다. 피해자는 피해 사실을 신고하고, 필요한 지원에 대해 상담함으로써 보다 신속한 도움을 받을 수 있습니다.  \n\n추가 문의 사항